In [1]:
import pandas as pd
import os
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time
import math
np.set_printoptions(threshold=sys.maxsize)

In [2]:
assignmentPath = os.sys.path[0]
data_xls = pd.read_excel(assignmentPath+'/GOV-ZA.50000ParallelleEnWoorde.nr.pos.full.xls', dtype=str, index_col=None)
data_xls.to_csv(assignmentPath+'/GOV-ZA.50000ParallelleEnWoorde.nr.pos.full.csv', encoding='utf-8', index = None, header=True)
data_df = pd.read_csv(assignmentPath+'/GOV-ZA.Toetsteks.5000ParallelleEnWoorde.nr.pos.full.csv')


In [3]:
# Identify the end of a sentence and add the stop symbol
# Remove all punctuation from the dataset 
period = data_df[ data_df['Token'] == '.' ].index
question = data_df[ data_df['Token'] == '?' ].index
esc_mark = data_df[ data_df['Token'] == '!' ].index
for key, value in data_df.POS.iteritems():
    if key in period:
        data_df['POS'][key] = '.'
    elif key in question:
        data_df['POS'][key] = '.'
    elif key in esc_mark:
        data_df['POS'][key] = '.'
punct = data_df[ data_df['POS'] == 'PUNCT' ].index
data_df.drop(punct, inplace=True)

In [4]:
# Preprocess the data and add <UNK> charector for all unknown words and default POS to V
data_df['POS'] = data_df['POS'].fillna('<UNK>')
data_df['Token'] = data_df['Token'].fillna('<UNK>')

In [5]:
#transform dataframe to a list of tuple (word,POS)
data_df = list(data_df.to_records(index=False))

In [6]:
train_data, test_dev_data = train_test_split(data_df, test_size=.20,random_state = 42)

In [7]:
test_data, dev_data = train_test_split(test_dev_data, test_size=0.5,random_state = 42)

In [8]:
tags = set(tags for words, tags in train_data)
vocab = set(words for words, tags in train_data)

In [9]:
# compute Emission Probability
def word_given_tag(word, tag, train_bag = train_data):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)#total number of times the passed tag occurred in train_bag
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    #now calculate the total number of times the passed word occurred as the passed tag.
    count_w_given_tag = len(w_given_tag_list) 
    return (count_w_given_tag, count_tag)

In [10]:
# compute  Transition Probability
def tag2_given_tag1(t2, t1, train_bag = train_data):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1]) + len(train_bag) # added 1 for smoothing 
    count_t2_t1 = 1 # initial was 0 added 1 for smoothing 
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [11]:
# creating t x t transition matrix of tags, t= no of tags
# Matrix(i, j) represents P(jth tag after the ith tag)
tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = tag2_given_tag1(t2, t1)[0]/tag2_given_tag1(t2, t1)[1]
print(tags_matrix)

[[0.00621302 0.00029586 0.00029586 0.00325444 0.00059172 0.00029586
  0.00059172 0.00029586 0.00029586 0.00118343 0.01982249 0.00266272
  0.00088757 0.00857988 0.00029586 0.00295858 0.00029586 0.00029586
  0.00029586 0.00147929 0.00088757 0.00207101 0.00029586 0.00029586
  0.00029586 0.00029586 0.00029586 0.00029586 0.00029586 0.00029586
  0.00059172 0.00029586 0.00088757 0.00059172 0.00029586 0.00029586
  0.00029586 0.00029586 0.0035503  0.00029586 0.00029586 0.00088757
  0.00059172 0.00059172 0.00029586 0.00680473 0.00414201 0.00059172
  0.00029586 0.00059172 0.00029586 0.00029586 0.00147929 0.00118343
  0.00029586 0.00118343 0.00059172 0.00029586 0.00029586 0.00029586
  0.00029586 0.00029586 0.00177515 0.00029586 0.00059172]
 [0.00031766 0.00031766 0.00031766 0.00031766 0.00031766 0.00031766
  0.00031766 0.00031766 0.00031766 0.00031766 0.00031766 0.00031766
  0.00031766 0.00031766 0.00031766 0.00031766 0.00031766 0.00031766
  0.00031766 0.00031766 0.00031766 0.00031766 0.00031766 0

In [12]:
# convert the matrix to a df for better readability
#the table is same as the transition table shown in section 3 of article
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))
display(tags_df)

,.,ADJ04,QUANT06,LOC,COP,REL06,M,ADJ00,A,N09,...,CONJ,ADJ01,ADJ09,REL05,POS06,REL10,POS01,N02,QUANT04,POS03
.,0.006213,0.000296,0.000296,0.003254,0.000592,0.000296,0.000592,0.000296,0.000296,0.001183,...,0.001183,0.000592,0.000296,0.000296,0.000296,0.000296,0.000296,0.001775,0.000296,0.000592
ADJ04,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,...,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318
QUANT06,0.000317,0.000317,0.000317,0.000317,0.000317,0.000317,0.000317,0.000317,0.000317,0.000317,...,0.000317,0.000317,0.000317,0.000317,0.000317,0.000317,0.000317,0.000317,0.000317,0.000317
LOC,0.001850,0.000308,0.000617,0.000617,0.000925,0.000308,0.000308,0.000308,0.000308,0.001542,...,0.000617,0.000308,0.000308,0.000308,0.000308,0.000308,0.000308,0.001233,0.000308,0.000308
COP,0.000631,0.000315,0.000315,0.000315,0.000315,0.000315,0.000315,0.000315,0.000315,0.000946,...,0.000315,0.000315,0.000315,0.000315,0.000315,0.000315,0.000315,0.001261,0.000315,0.000315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
REL10,0.000635,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,...,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318
POS01,0.000635,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,...,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318
N02,0.000937,0.000312,0.000312,0.000625,0.000625,0.000312,0.000312,0.000312,0.000312,0.000937,...,0.000625,0.000625,0.000312,0.000312,0.000312,0.000312,0.000312,0.000312,0.000312,0.000312
QUANT04,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,...,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318,0.000318


In [13]:
def Viterbi(words, train_bag = train_data):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
     
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
             
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [14]:
test_run = [i for i in dev_data]
test_run_base = [sent for sent in test_run]
test_untagged_words = [sent[0] for sent in test_run]

In [15]:
#Here We will only test sentences to check the accuracy
#as testing the whole training set takes huge amount of time
start = time.time()
tagged_seq = Viterbi(test_untagged_words)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)
 
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if tuple(i) == tuple(j)] 
 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)


Time taken in seconds:  107.00290703773499
Viterbi Algorithm Accuracy:  49.746192893401016


In [16]:
test = [i for i in test_data]
test_base = [sent for sent in test_run]
tests_untagged_words = [sent[0] for sent in test_run]

In [17]:
#Here We will only test sentences to check the accuracy
#as testing the whole training set takes huge amount of time
start = time.time()
tagged_seq = Viterbi(tests_untagged_words)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)
 
# accuracy
check = [i for i, j in zip(tagged_seq, test_base) if tuple(i) == tuple(j)] 
 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)


Time taken in seconds:  108.68649411201477
Viterbi Algorithm Accuracy:  49.746192893401016


In [18]:
test_sent="iforomo lesibawo namkha lifumane e-ofisini eliseduze le-SARS ."
pred_tags_withoutRules= Viterbi(test_sent.split(" "))
print(pred_tags_withoutRules)


# Khupha      V
# 1          iforomo    DEM
# 2         lesibawo    DEM
# 3           namkha   CONJ
# 4         lifumane      V
# 5        e-ofisini    LOC
# 6        eliseduze    REL
# 7          le-SARS    DEM
# 8                .      .


[('iforomo', 'DEM'), ('lesibawo', 'DEM'), ('namkha', 'CONJ'), ('lifumane', 'V'), ('e-ofisini', 'LOC'), ('eliseduze', '.'), ('le-SARS', 'DEM'), ('.', '.')]


In [19]:
test_sent="<UNK> Iforomo lesibawo kufanele lizaliswe mbawi ingasi omunye umuntu nakungasi njalo isibawo asizukutjhejwa ."
pred_tags_withoutRules= Viterbi(test_sent.split(" "))
print(pred_tags_withoutRules)

#   <UNK>      V
# 10         Iforomo    DEM
# 11        lesibawo    DEM
# 12        kufanele      V
# 13       lizaliswe      V
# 14           mbawi      N
# 15          ingasi      V
# 16          omunye  ADJ01
# 17          umuntu    N01
# 19       nakungasi      V
# 20           njalo    ADV
# 21         isibawo      N
# 22  asizukutjhejwa      V
# 23               .      .

[('<UNK>', '<UNK>'), ('Iforomo', 'DEM'), ('lesibawo', 'DEM'), ('kufanele', 'V'), ('lizaliswe', 'V'), ('mbawi', '.'), ('ingasi', 'V'), ('omunye', 'ADJ01'), ('umuntu', 'N01'), ('nakungasi', 'V'), ('njalo', 'ADV'), ('isibawo', 'N'), ('asizukutjhejwa', 'V'), ('.', '.')]
